In [27]:
import numpy as np
import pandas as pd
import datetime

In [28]:
dataset=pd.read_csv("../dataset.csv",sep=";")
dataset.drop(["node"],inplace=True,axis=1)
dataset["date"]=pd.to_datetime(dataset["date"],format="%Y-%m-%d %H:%M:%S")
train=dataset.query("date.dt.year>2017 and date.dt.year <2022")
train=train.set_index("date")
test=dataset.query("date.dt.year==2022")
test=test.set_index("date")

In [29]:
t_media=pd.read_csv("../temperaturas/DMC_330020_Quinta_Normal_Santiago_media.csv",sep=";")
t_media["date"]=pd.to_datetime(t_media["date"],format="%Y-%m-%d %H:%M:%S")
t_media=t_media.set_index("date")
fin= datetime.datetime.strptime("2023-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
t_media_fin=t_media.loc[fin]
t_media=t_media.query("date.dt.year>2017 and date.dt.year<2023")
t_media.loc[fin]=t_media_fin
t_media_hora=t_media.resample("1h").ffill()
t_media_train=t_media_hora.query("date.dt.year>2017 and date.dt.year<2022")
t_media_test=t_media_hora.query("date.dt.year==2022")

In [30]:
subestaciones=["AJAHUEL","BUIN","CHENA","CNAVIA","ELSALTO","FLORIDA","LOSALME"]
for subestacion in subestaciones:
    filtro="substation=='{0}'".format(subestacion)
    ds_train=train.query(filtro)["consumption"].groupby("date").aggregate(consumption="mean")
    ds_test=test.query(filtro)["consumption"].groupby("date").aggregate(consumption="mean")
    deltas = 365*24*4
    #agregamos un día al rango por el año bisiesto 2020
    deltas+=24
    start = datetime.datetime.strptime("2018-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
    train_full = [start + datetime.timedelta(hours=x) for x in range(deltas)]
    deltas = 365*24
    start = datetime.datetime.strptime("2022-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
    test_full = [start + datetime.timedelta(hours=x) for x in range(deltas)]
    missing_train=[]
    for fecha in train_full:
        if not (fecha in ds_train.index):
            missing_train.append(fecha)
    missing_test=[]
    for fecha in test_full:
        if not (fecha in ds_test.index): 
            missing_test.append(fecha)
    for missing in missing_test:
        delta=1
        hora_previa=missing-datetime.timedelta(hours=delta)
        while(not (hora_previa in ds_test.index)):
            delta=delta+1
            hora_previa=missing-datetime.timedelta(hours=delta)
        delta=1
        hora_siguiente=missing+datetime.timedelta(hours=delta)
        while(not (hora_siguiente in ds_test.index)):
            delta=delta+1
            hora_siguiente=missing+datetime.timedelta(hours=delta)

        consumo_previo=ds_test.loc[hora_previa]["consumption"]
        consumo_siguiente=ds_test.loc[hora_siguiente]["consumption"]
        consumo_faltante=(consumo_previo+consumo_siguiente)/2
        ds_test.loc[missing]=[consumo_faltante] 
    for missing in missing_train:
        delta=1
        hora_previa=missing-datetime.timedelta(hours=delta)
        while(not (hora_previa in ds_train.index)):
            delta=delta+1
            hora_previa=missing-datetime.timedelta(hours=delta)
        delta=1
        hora_siguiente=missing+datetime.timedelta(hours=delta)
        while(not (hora_siguiente in ds_train.index)):
            delta=delta+1
            hora_siguiente=missing+datetime.timedelta(hours=delta)
            
        consumo_previo=ds_train.loc[hora_previa]["consumption"]
        consumo_siguiente=ds_train.loc[hora_siguiente]["consumption"]
        consumo_faltante=(consumo_previo+consumo_siguiente)/2
        ds_train.loc[missing]=[consumo_faltante]
    ds_train.sort_index(inplace=True)
    ds_test.sort_index(inplace=True)
    ds_train=ds_train.join(t_media_train["t_media"])
    ds_train["w"]=ds_train.index.dayofweek
    ds_train["h"]=ds_train.index.hour
    ds_test=ds_test.join(t_media_test["t_media"])
    ds_test["w"]=ds_test.index.dayofweek
    ds_test["h"]=ds_test.index.hour
    ds_train.to_csv("train_hora_{0}.csv".format(subestacion))
    ds_test.to_csv("test_hora_{0}.csv".format(subestacion))